In [81]:
import chainer
from chainer import training, iterators, optimizers, serializers, Chain
import chainer.functions as F
import chainer.links as L
 
from chainer.training import extensions
from chainer.datasets import tuple_dataset
from chainer import optimizers, Chain, dataset, datasets, iterators
import numpy as np

In [82]:
def data_read( file_name, key):
    teachers = np.array([] )
    answers =  np.array([] )

    f = open( file_name, mode = "r" )
    f_string = f.readlines()
    data = np.array([] )
    print("test")
    
    for i in range( 0, len( f_string ) ):
        #引数を用いて正解ラベルを振り分ける
        tmp_data = np.zeros(10)
        for j in f_string[i].replace( "\n", "" ):
                tmp_data[int(j)] += 1
                
        if( i < key-1):
                data = np.append( data, tmp_data )
        else:
            teachers = np.append( teachers, data )
            answers = np.append( answers, tmp_data )
            data = np.delete( data, 0 )
            data = np.append( data, tmp_data )
        
        if( (i + 2)%100 == 0 ):
            print(i/len(f_string))
        #if( i != 0 and i%key == 0 ):
            #teachers = np.append( teachers, data)
            #data = np.array([] )
            #for j in f_string[i].replace( "\n", "" ):
                #tmp_data[int(j)] += 1
            #answers = np.append( answers, tmp_data )
        #else:
            #for j in f_string[i].replace( "\n", "" ):
                #tmp_data[int(j)] += 1
                #data = np.append( data, tmp_data )
    
    f.close()
    
    teachers = teachers.astype( np.float32 )
    answers = answers.astype( np.float32 )
    
    teachers = np.reshape( teachers, ( int( len( teachers ) / 10 / key ), key, 10 ) )
    
    answers = np.reshape( answers, ( int( len( answers ) / 10 ) , 10 ) )
    return teachers, answers


In [ ]:
teachers, answers = data_read( 'numbers.txt', 5)

test
0.017632241813602016
test
0.035624325296869375
test
0.05361640878013674
test
0.0716084922634041
test
0.08960057574667146
test
0.10759265922993883
test
0.1255847427132062
test
0.14357682619647355
test
0.1615689096797409
test
0.17956099316300828
test
0.19755307664627564
test
0.215545160129543
test
0.23353724361281036
test
0.2515293270960777
test
0.2695214105793451
test
0.28751349406261245
test
0.3055055775458798
test
0.3234976610291472
test
0.34148974451241454
test
0.3594818279956819
test
0.37747391147894926
test
0.3954659949622166
test
0.413458078445484
test
0.43145016192875135
test
0.4494422454120187
test
0.4674343288952861
test
0.48542641237855344
test
0.5034184958618207
test
0.5214105793450882
test
0.5394026628283555
test
0.5573947463116229
test
0.5753868297948902
test
0.5933789132781576
test
0.6113709967614249
test
0.6293630802446923
test
0.6473551637279596
test
0.6653472472112271
test
0.6833393306944944
test
0.7013314141777618
test
0.7193234976610291
test
0.7373155811442965
te

In [1]:
print(len(teachers))
print(len(answers))
print(teachers[0])
print(answers[0])

NameError: name 'teachers' is not defined

In [68]:
#ニューラルネットワークの構築。
class RNN(Chain):
    
    def __init__(self, n_hidden, n_output):
        super(RNN, self).__init__(
            l1=L.LSTM(None, n_hidden),
            l2=L.Linear(None, n_output),
        )
        
    def reset_state(self):
        self.l1.reset_state()
        
    def __call__(self, x, t):
        y = self.predict(x)
        loss = F.mean_squared_error(y, t)
        data = y.data[:]
        accuracy = self.accuracy(data, t)
        chainer.reporter.report({'accuracy':accuracy},self)
        chainer.reporter.report({'loss':loss},self)
        return loss
    
    def accuracy(y, t):
        correct = 0
        for j in range( 0, len(y) ):
            if y[j].size:
                for i in range( 0, 3 ):
                    if( t[j][y[j].argmax()] ):
                        correct += 1
                    y[j][y[j].argmax()]= np.amin(y[j])
        return correct / (len(y) * 3)
        
    
    def predict(self, x):
        if train:
            h1 = F.dropout(self.l1(x),ratio = 0.5)
        else:
            h1 = self.l1(x)
        return self.l2(h1)

In [69]:
#Updaterを拡張する
from chainer import Variable, reporter

class LSTMUpdater(training.StandardUpdater):
    def __init__(self, data_iter, optimizer, device=None):
        super(LSTMUpdater,self).__init__(data_iter, optimizer, device=None)
        self.device = device
        
    def update_core(self):
        data_iter = self.get_iterator("main")
        optimizer = self.get_optimizer("main")
        
        batch = data_iter.__next__()
        x_batch, t_batch = chainer.dataset.concat_examples(batch, self.device)
        
        optimizer.target.reset_state()           
        optimizer.target.cleargrads()
        #loss = optimizer.target(Variable(x_batch), Variable(t_batch))
        loss = optimizer.target(x_batch, t_batch)
        loss.backward()
        loss.unchain_backward()                  
        optimizer.update() 


In [70]:
#　教師データのtupleを作成する
data = list(zip(teachers, answers))
N = len(data)
n_batchsize = 30
n_epoch = 1

#モデルを使う準備。オブジェクトを生成
n_hidden = 10
n_output = 10
model = RNN(n_hidden, n_output)
optimizer = optimizers.Adam()
optimizer.setup(model)

#学習用データと検証用データに分ける
train, test = chainer.datasets.split_dataset_random(data, int(N * 0.8))
train_iter = chainer.iterators.SerialIterator(train, n_batchsize, shuffle=False)
test_iter = chainer.iterators.SerialIterator(test, n_batchsize, repeat=False, shuffle=False)
updater = LSTMUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (n_epoch, "epoch"), out="result")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ["epoch", "main/loss", "validation/main/loss", "main/accuracy", "validation/main/accuracy", "elapsed_time"])) # エポック、学習損失、テスト損失、学習正解率、テスト正解率、経過時間
trainer.extend(extensions.PlotReport(['main/loss', 'val/main/loss'], x_key='epoch', file_name='loss.png'))
trainer.extend(extensions.PlotReport(['main/accuracy', 'val/main/accuracy'], x_key='epoch', file_name='accuracy.png'))
trainer.run()

Exception in main training loop: accuracy() takes 2 positional arguments but 3 were given
Traceback (most recent call last):
  File "/home/rurito/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/chainer/training/trainer.py", line 316, in run
    update()
  File "/home/rurito/.pyenv/versions/anaconda3-4.3.1/lib/python3.6/site-packages/chainer/training/updaters/standard_updater.py", line 175, in update
    self.update_core()
  File "<ipython-input-69-d1b759a50fe5>", line 19, in update_core
    loss = optimizer.target(x_batch, t_batch)
  File "<ipython-input-68-75f5dd9d3e59>", line 17, in __call__
    accuracy = self.accuracy(data, t)
Will finalize trainer extensions and updater before reraising the exception.


TypeError: accuracy() takes 2 positional arguments but 3 were given

In [27]:
def accuracy(y, t):
        correct = 0
        for j in range( 0, len(y) ):
            if y[j].size:
                for i in range( 0, 3 ):
                    if( t[j][y[j].argmax()] ):
                        correct += 1
                    y[j][y[j].argmax()]= np.amin(y[j])
        print(correct)
        return correct / (len(y) * 3)

y = np.array([[ 2.32572779e-02, 1.50612950e-01, -7.73513615e-02,
            1.24777101e-01, 1.66107967e-01, 4.45912518e-02,
            3.39158834e-03, -3.49103138e-02, 6.63672294e-03,
           -9.04915929e-02],
            [ 2.32572779e-02, 1.50612950e-01, -7.73513615e-02,
            1.24777101e-01, 1.66107967e-01, 4.45912518e-02,
            3.39158834e-03, -3.49103138e-02, 6.63672294e-03,
           -9.04915929e-02]])
t = np.array([[0,1,0,1,1,0,0,0,0,0],[0,1,0,1,1,0,0,0,0,0]])
accuracy(y,t)

5


0.8333333333333334